In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer # accept mixed types
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split , GridSearchCV

In [2]:
data = pd.read_csv('train.csv')
data.head(2)

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,-1,61518,19297675,-1,6,928106152,2541203883,3606287998,306394936,2967512603,...,0,0,0,0,0,0,0,1083418268,3831440054,830706566
1,-1,356604,19320277,-1,12,3382400190,2018710401,1302581498,2308170622,2967512603,...,2,0,0,0,6,0,0,2993954402,2379633508,1284343215


In [3]:
# rows: samples
data.columns

Index(['label', 'id', 'visitTime', 'purchaseTime', 'hour', 'C1', 'C2', 'C3',
       'C4', 'C5', 'C6', 'C7', 'C8', 'N1', 'C9', 'N2', 'N3', 'N4', 'N5', 'N6',
       'N7', 'N8', 'N9', 'N10', 'C10', 'C11', 'C12'],
      dtype='object')

In [4]:
from collections import Counter
a=data['label']
Counter(a)

Counter({-1: 31371, 1: 57})

In [5]:
data.dropna(inplace=True)# remove rows with null values

In [6]:
c4 = pd.get_dummies(data["C4"], drop_first=True)
c4

,652481754,2210625817,2308170622
0,0,0,0
1,0,0,1
2,0,1,0
3,1,0,0
4,0,1,0
...,...,...,...
31423,1,0,0
31424,1,0,0
31425,1,0,0
31426,1,0,0


In [7]:
c5 = pd.get_dummies(data["C5"], drop_first=True)
c5

,2967512603,3955179140
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
31423,1,0
31424,1,0
31425,1,0
31426,1,0


In [8]:
data=data.drop(["id",'purchaseTime','C1','C2','C3','C6','C7','C8','C9','C10','C11','C12'], axis=1)
data.head(2)

,label,visitTime,hour,C4,C5,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10
0,-1,19297675,6,306394936,2967512603,0,0,0,0,0,0,0,0,0,0
1,-1,19320277,12,2308170622,2967512603,1,0,4,2,0,0,0,6,0,0


In [9]:
# We create the preprocessing pipelines for both numeric and categorical data.
X = data.drop("label", axis=1) # our features
y = data["label"] # labels

numeric_features = ['N1', 'N2', 'N3', 'N4', 'N5', 'N6',
       'N7', 'N8', 'N9', 'N10']

# 1st pipeline for our numerical feat.
numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='median')), # for the missing values, we replace them by the median
    ('scaler', StandardScaler())])

# for our cat. features
categorical_features = ['C4','C5']
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# combine the two pipelines
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
log_reg_pipe = Pipeline(steps = [('preprocessor', preprocessor), # preprocessing
                      ('classifier', LogisticRegression())]) # our model

X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2,random_state=42)

log_reg_pipe.fit(X_train, y_train)

test_score = log_reg_pipe.score(X_test, y_test)
print("model score:  %.3f " % test_score)

model score:  0.998 


/Users/elainelim/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
from imblearn.over_sampling import RandomOverSampler

In [11]:
class_count_0, class_count_1 = data['label'].value_counts()

# Separate class
class_0 = data[data['label'] == -1]
class_1 = data[data['label'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (31371, 15)
class 1: (57, 15)


In [12]:
ros = RandomOverSampler(random_state=42)

# fit predictor and target variable
X_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({-1: 31371, 1: 57})
Resample dataset shape Counter({-1: 31371, 1: 31371})


In [13]:
X_ros_train , X_ros_test , y_ros_train , y_ros_test = train_test_split(X_ros, y_ros, test_size=0.2,random_state=42)

model=log_reg_pipe.fit(X_ros_train, y_ros_train)


/Users/elainelim/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
test_score = model.score(X_ros_test, y_ros_test)
print("model score:  %.3f " % test_score)

model score:  0.927 


In [15]:
test = pd.read_csv('test.csv')
test.head(2)

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,-1,25540,19420446,-1,16,2569487005,2018710401,1429850909,2308170622,2967512603,...,0,0,0,0,0,0,0,1887882016,3617851689,4220791646
1,-1,29938,19431062,-1,19,1981742921,1235983246,4203734567,652481754,2116767313,...,0,0,0,0,0,0,0,3375463994,2379633508,1284343215


In [16]:
test_set=test.drop(["id",'purchaseTime','C1','C2','C3','C6','C7','C8','C9','C10','C11','C12'], axis=1)
test_set.head(2)

,label,visitTime,hour,C4,C5,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10
0,-1,19420446,16,2308170622,2967512603,0,0,0,0,0,0,0,0,0,0
1,-1,19431062,19,652481754,2116767313,0,0,0,0,0,0,0,0,0,0


In [17]:
test_x=test_set.drop(["label"],axis=1)

In [18]:
prob=model.predict_proba(test_x)
prob

array([[0.74963897, 0.25036103],
       [0.89513989, 0.10486011],
       [0.89513989, 0.10486011],
       ...,
       [0.80977064, 0.19022936],
       [0.80977064, 0.19022936],
       [0.89513989, 0.10486011]])

In [19]:
prob = pd.DataFrame(data=prob,columns=["prob_neg", "prob_pos"])
prob.head(5)

,prob_neg,prob_pos
0,0.749639,0.250361
1,0.895140,0.104860
2,0.895140,0.104860
3,0.895140,0.104860
4,0.809771,0.190229


In [20]:
test_prob=data = pd.concat([test['id'],prob], axis=1)
test_prob.head(5)

,id,prob_neg,prob_pos
0,25540,0.749639,0.250361
1,29938,0.895140,0.104860
2,9585,0.895140,0.104860
3,23597,0.895140,0.104860
4,27695,0.809771,0.190229


In [21]:
test_prob.to_csv('test_with_proba.csv',index = False, header=True)